## Capture images

In [ ]:
import time
import cv2
import datetime as dt
from timelapse.utils import calculate_brightness, find_correct_exposure, get_properties, \
find_correct_exposure, find_best_focus, calculate_sharpness
from notify_run import Notify
notify = Notify()

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s',
                    filename='timelapse.log', 
                    filemode='w', 
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)
logging.info('Started')

In [ ]:
starttime = time.time()
notification_sent = False
exposure = 62
focus = 999

while True:
    try:
        # FIND OPTIMAL PARAMETERS
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)
        cap.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
        cap, exposure, n_iter = find_correct_exposure(cap,100,125,1000, verbose = True)
        cap, new_focus = find_best_focus(cap)
        if abs(focus-new_focus)>10:
            focus=new_focus
        
        # REINITIALIZE THE CAMERA
        cap.release()
        cv2.destroyAllWindows()
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 1)
        cap.set(cv2.CAP_PROP_EXPOSURE, exposure)
        cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
        cap.set(cv2.CAP_PROP_FOCUS, focus)

        # TAKE A PICTURE AND STORE IT
        timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        ret,frame = cap.read()
        props = get_properties(cap)
        cv2.imwrite(f'images/capture_{timestamp}.png',frame)
        cap.release()
        cv2.destroyAllWindows()
        logging.info(f'capture_{timestamp}.png ---')
        logging.info(f'exposure: {exposure}, brightness: {calculate_brightness(frame):.2f}')
        logging.info(f'focus: {focus}, sharpness: {calculate_sharpness(frame):.2f}')
        logging.info(f'props: {props}\n\n')
        
    except:
        if not notification_sent:
            notify.send('Timelapse has run into an error')
            notification_sent=True
        try:
            cap.release()
            cv2.destroyAllWindows()
        except:
            pass

        
    time.sleep(60.0 - ((time.time() - starttime) % 60.0))